In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

image_dir = 'dataset/balanced_dataset'

# Define image size for resizing
image_size = (64, 64)  # Fixed size for consistency

# Load and preprocess images
def load_images_and_labels(data, image_dir, image_size):
    images = []
    labels = []
    for _, row in data.iterrows():
        image_path = os.path.join(image_dir, row['image_name'])
        if os.path.exists(image_path):
            # Load and resize image
            img = load_img(image_path, target_size=image_size)
            img_array = img_to_array(img) / 255.0  # Normalise pixel values
            images.append(img_array)
            labels.append(row['target'])
        else:
            print(f"Image not found: {image_path}")
    return np.array(images), np.array(labels)

data = pd.read_csv('dataset/balanced_metadata.csv')
images, labels = load_images_and_labels(data, image_dir, image_size)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
labels_one_hot = to_categorical(labels, num_classes)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)

# Build a Fully Connected Neural Network (FCNN)
model = Sequential([
    Flatten(input_shape=image_size + (3,)),  # Flatten input
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Output layer for multiclass classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.2962 - loss: 1.5100 - val_accuracy: 0.4056 - val_loss: 1.2739
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.3705 - loss: 1.3136 - val_accuracy: 0.4863 - val_loss: 1.1840
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.3724 - loss: 1.2739 - val_accuracy: 0.4688 - val_loss: 1.1619
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.4064 - loss: 1.2128 - val_accuracy: 0.4437 - val_loss: 1.1819
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.3970 - loss: 1.2079 - val_accuracy: 0.4275 - val_loss: 1.1600
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.3957 - loss: 1.2003 - val_accuracy: 0.4806 - val_loss: 1.1221
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.4037 - loss: 1.1789 - val_accuracy: 0.4725 - val_loss: 1.0912
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.4090 - loss: 1.1684 - val_accu

In [4]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array


# Directory for images
image_dir = 'dataset/balanced_dataset'
image_size = (64, 64)  # Fixed size for consistency

# Load and preprocess images
def load_images_and_labels(data, image_dir, image_size):
    images = []
    labels = []
    for _, row in data.iterrows():
        image_path = os.path.join(image_dir, row['image_name'])
        if os.path.exists(image_path):
            img = load_img(image_path, target_size=image_size)
            img_array = img_to_array(img) / 255.0  # Normalise pixel values
            images.append(img_array)
            labels.append(row['target'])
        else:
            print(f"Image not found: {image_path}")
    return np.array(images), np.array(labels)

# Load dataset
data = pd.read_csv('dataset/balanced_metadata.csv')
images, labels = load_images_and_labels(data, image_dir, image_size)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
labels_one_hot = to_categorical(labels, num_classes)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)

# Function to create model
def create_model(learning_rate=0.001, neurons=128, dropout_rate=0.5):
    model = Sequential([
        Flatten(input_shape=image_size + (3,)),
        Dense(neurons, activation='relu'),
        Dropout(dropout_rate),
        Dense(neurons // 2, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Wrap the model for GridSearchCV
model = KerasClassifier(
    model=create_model,
    learning_rate=0.001,  # Default hyperparameter
    neurons=128,          # Default hyperparameter
    dropout_rate=0.5,     # Default hyperparameter
    verbose=0
)

# Define the grid of hyperparameters to search
param_grid = {
    'learning_rate': [0.001, 0.01],
    'neurons': [64, 128],
    'dropout_rate': [0.3, 0.5],
    'batch_size': [16, 32],
    'epochs': [5, 10]
}

# Perform Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Output the best parameters and performance
print(f"Best Parameters: {grid_result.best_params_}")
print(f"Best Accuracy: {grid_result.best_score_ * 100:.2f}%")

# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
test_loss, test_accuracy = best_model.model_.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Fitting 3 folds for each of 32 candidates, totalling 96 fits


2024-12-09 11:33:39.833633: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 11:33:39.834266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 11:33:39.834705: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 11:33:39.836057: I tensorflow/core/platform/cpu_featu

KeyboardInterrupt: 